In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
import random

In [4]:
X = pd.read_csv('combined_data.csv')
X = X.sample(frac=1, random_state=42).reset_index(drop=True)
y=X['Class']
X = X.drop('Class', axis=1)

#print(X.head(4))
#print(y.head(4))

In [7]:
SEED=42
n_folds=5
clf = LogisticRegression(max_iter=1000)

cv = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

score = cross_validate(clf, X=X, y=y, cv=cv, return_train_score=True, return_estimator=True, scoring = 'roc_auc')
#print("This is the score object:")
#print (score)
print("Average AUC test set:", np.mean(score['test_score']))


Average AUC test set: 0.583968253968254


In [8]:
from sklearn.metrics import recall_score

sensitivities = []
specificities = []

for i, (train_index, test_index) in enumerate(cv.split(X, y)):
    # Get the fitted estimator for the current fold
    fold_estimator = score['estimator'][i]

    # Get the test data for the current fold
    X_test_fold = X.iloc[test_index]
    y_test_fold = y.iloc[test_index]

    # Make predictions on the test set for the current fold
    y_pred_fold = fold_estimator.predict(X_test_fold)

    # Calculate sensitivity (recall for class 1)
    sensitivity = recall_score(y_test_fold, y_pred_fold, pos_label=1)
    sensitivities.append(sensitivity)

    # Calculate specificity (recall for class 0)
    specificity = recall_score(y_test_fold, y_pred_fold, pos_label=0)
    specificities.append(specificity)

print("Average Sensitivity after cross-validation:", np.mean(sensitivities))
print("Average Specificity after cross-validation:", np.mean(specificities))

Average Sensitivity after cross-validation: 0.275
Average Specificity after cross-validation: 0.6454761904761904
